In [ ]:
#Importing libraries
import requests as rq
import json
from pymongo import MongoClient
import pandas as pd

try:
    #Creating MongoDB schema and collection for raw data
    client = MongoClient('localhost',27017)
    db_name = 'Education_NY'
    db = client[db_name]
    collection = db['edu_ny_rawdata']
    
    #Downloading the data from newyork state government website
    data_url = 'https://data.ny.gov/resource/64ar-ackz.json'
    edu_year_list = ['Fall 2016','Fall 2017','Fall 2018','Fall 2019']
    for eyl in edu_year_list:
        #Retrieving data from API
        req_url = data_url+'?term='+eyl+'&$limit=5000'
        response = rq.get(req_url)
        #response.raise_for_status()
        
        #Inserting the raw data into MongoDB
        if (response.status_code == 200):
            print("The request was a success!")
            json_data = response.json()
            l_count=len(json_data)
            collection.insert_many(json_data)
            print(l_count)
        elif (response.status_code == 404):
            print("Result not found!")
    
except requests.exceptions.HTTPError as errh:
    print(errh)
except requests.exceptions.ConnectionError as errc:
    print(errc)
except requests.exceptions.Timeout as errt:
    print(errt)
except requests.exceptions.RequestException as err:
    print(err)

In [ ]:
#Data Exploration
# Geo location data from all records is removed
collection.update_many({}, {'$unset': {'location_1': None}})

#Loading data into dataframe
df = pd.DataFrame(list(collection.find()))

# shape of the data
print('==========================')
print('**************************')
print('Shape of dataset')
print('**************************')
print(df.shape)
print('==========================')

# preview dataset
print('==========================')
print('**************************')
print('Basic Preview')
print('**************************')
print(df.head())
print('==========================')

# view column names
print('==========================')
print('**************************')
print('Column Names of dataset')
print('**************************')
print(df.columns)
print('==========================')

# view dataframe summary
print('==========================')
print('**************************')
print('Brief summary about dataset')
print('**************************')
print(df.info())
print('==========================')

print('Number of Categorical Columns: ', len(df.select_dtypes(include=object).columns))
print('Number of Numerical Columns: ', len(df.select_dtypes(exclude=object).columns))

In [ ]:
# Data Preprocessing
# check for null values
print(df.isnull().sum())

In [ ]:
# check for missing values
print(df.isna().sum())

In [ ]:
#Replacing term column values with numeric years
df['term'].replace({"Fall 2016":"2016", "Fall 2017":"2017", "Fall 2018":"2018", "Fall 2019":"2019"}, inplace=True)

print(df['term'].head())

In [ ]:
#Replacing student count of <4 values with 1(This assumption is to make the data uniform for processing)
df['total_attending_suny_institutions'].replace({"<4":"1"}, inplace=True)
df['attending_suny_doctoral_institutions'].replace({"<4":"1"}, inplace=True)
df['attending_suny_comprehensive_colleges'].replace({"<4":"1"}, inplace=True)
df['attending_suny_technology_colleges'].replace({"<4":"1"}, inplace=True)
df['attending_suny_community_colleges'].replace({"<4":"1"}, inplace=True)



In [ ]:
df.info()

In [ ]:
# Converting datatype from object to integer to get summary statistics and perform aggregation

df['total_attending_suny_institutions'] = df['total_attending_suny_institutions'].astype('int')
df['attending_suny_doctoral_institutions'] = df['attending_suny_doctoral_institutions'].astype('int')
df['attending_suny_comprehensive_colleges'] = df['attending_suny_comprehensive_colleges'].astype('int')
df['attending_suny_technology_colleges'] = df['attending_suny_technology_colleges'].astype('int')
df['attending_suny_community_colleges'] = df['attending_suny_community_colleges'].astype('int')

In [ ]:
df.info()

In [ ]:
# Getting the list of unique counties to find total number of enrolled students in each county

unique_county_list = df['county'].unique().tolist()
print(unique_county_list)

In [ ]:
len(unique_county_list)

In [ ]:
# Converting the dataframe to csv file for storing in Postgres database
df.to_csv(r'education_ny.csv', index=False, header=True)

In [ ]:
#Create table

try:
    dbConnection = psycopg2.connect(
        user = "postgres",
        password = "lightyear",
        host = "localhost",
        port = "5432",
        database = "dap_group_m")
    dbConnection.set_isolation_level(0)
    dbCursor = dbConnection.cursor()
    dbCursor.execute("""
        CREATE TABLE edu_ny_data(
        _id varchar(255),
        term varchar(255),
        high_school_name varchar(255),
        high_school_ceeb varchar(255),
        city varchar(255),
        county varchar(255),
        zip_code varchar(255),
        total_attending_suny_institutions int,
        attending_suny_doctoral_institutions int,
        attending_suny_comprehensive_colleges int,
        attending_suny_technology_colleges int,
        attending_suny_community_colleges int
        );
    """)
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): 
        dbConnection.close()

In [ ]:
#Insert values
import psycopg2
import csv
try:
    dbConnection = psycopg2.connect(
        user = "postgres",
        password = "lightyear",
        host = "localhost",
        port = "5432",
        database = "dap_group_m")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    with open('education_ny.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip the header
        for row in reader:
            dbCursor.execute("INSERT INTO edu_ny_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",row)
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): 
        dbConnection.close()

In [ ]:
# Data Visualization on education dataset

